## Initialize the system

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

G=3 # number of networks
N=[4,4,4] # number of nodes in each network
K=2 # number of commodities
K_weight=[0.5,0.5]
F=2 # number of frequency bands

# matrix of queue lengths for each commodity
queues_k1=[np.zeros(N[0]),np.zeros(N[1]),np.zeros(N[2])]
queues_k2=[np.zeros(N[0]),np.zeros(N[1]),np.zeros(N[2])]

# matrix of intranetwork links
intranetwork_links_0=np.zeros((N[0],N[0]))
intranetwork_links_1=np.zeros((N[1],N[1]))
intranetwork_links_2=np.zeros((N[2],N[2]))
intra_links=[intranetwork_links_0,intranetwork_links_1, intranetwork_links_2]

# assume we have links 0->1->3 and 0->2->3 for all networks where 3 is the sink
for i in range(0,len(intra_links)):
    intra_links[i][0,1]=1
    intra_links[i][0,2]=1
    intra_links[i][1,3]=1
    intra_links[i][2,3]=1

# define transmission rate matrix 
# we are assuming that each link is on and can transmit one packet each time step
state=intra_links

# outside arrival probability matrix for each node in the networks (probability that a packet arrives)
# for each commodity 
arrivals_k1=[np.zeros(N[0]),np.zeros(N[1]),np.zeros(N[2])]
arrivals_k2=[np.zeros(N[0]),np.zeros(N[1]),np.zeros(N[2])]
for j in range(0,len(arrivals_k1)):
    arrivals_k1[j][0]=0.5
    arrivals_k2[j][0]=0.3

print("There are", G, "networks with", N, "nodes respectively")
print("There are", K, "commodities with",K_weight,"weights resptively")
print("There are", F, "frequency bands and all networks have the same notion of frequency")

There are 3 networks with [4, 4, 4] nodes respectively
There are 2 commodities with [0.5, 0.5] weights resptively
There are 2 frequency bands and all networks have the same notion of frequency


## Begin the Period

At the beginning of a period of length T, the interference links will be randomly assigned a 0 or 1 if interference will occur between nodes across networks.  This will not change during the period but the networks won't have access to it.
Also at the beginning of the period, all of the networks communicate something about what they are doing.
This will be stored in their knowledge base.
Each network will be using the backpressure algorithm to route variables.

In [13]:
# period length
T=10

# initialize the interference links matrix
# 0 means no interference, 1 means interference
interf_links_01=np.random.choice(2, (N[0],N[1]))
interf_links_02=np.random.choice(2, (N[0],N[2]))
interf_links_12=np.random.choice(2, (N[1],N[2]))
# the first is the interferences between networks 0 and 1
# the second is the interferences between networks 0 and 2
# the third is the interferences between networks 1 and 2
interf_links=[interf_links_01,interf_links_02, interf_links_12]
interf_nets=[(0,1),(0,2),(1,2)]

control=[np.zeros((N[0],N[0])),np.zeros((N[1],N[1])),np.zeros((N[2],N[2]))]
commodity=[np.zeros((N[0],N[0])),np.zeros((N[1],N[1])),np.zeros((N[2],N[2]))]
state=intra_links

# update the queue lengths at each time step
for time in range(0,T):
    # find each networks optimal control decision using backpressure
    # the control is how many packets to send along a link, commodity is which commodity to send
    for net in range(0,G):
        control[net], commodity[net]=backpressure(state[net],queues_k1[net],queues_k2[net])

    # choose which frequency to send along
    
    # random frequency generator - there are 2 frequencies freq=0 or 1
    # initialize the interference links matrix
    freq_net0=np.random.choice(2, (N[0],N[0]))
    freq_net1=np.random.choice(2, (N[1],N[1]))
    freq_net2=np.random.choice(2, (N[2],N[2]))
    freq=[freq_net0, freq_net1, freq_net2]
    
    # create an interference matrix where 1 is interference and 0 is not
    interference_freq1, interference_freq2=find_interferences(control,freq,interf_links,interf_nets)
    
    # move packets along unless there are interferences
    # packet_movement function updates the queues
    print("k1 queues",queues_k1[0])
    print("k2 queues",queues_k2[0])
    print("control taken",control[0])
    queues_k1, queues_k2=packet_movement(queues_k1, queues_k2,control,commodity, freq,interference_freq1, interference_freq2)
    
    # add the outside arrivals
    queues_k1=add_arrivals(queues_k1, arrivals_k1)
    queues_k2=add_arrivals(queues_k2, arrivals_k2)

print(queues_k1)
print(queues_k2)

k1 queues [ 2.  2.  1.  0.]
k2 queues [ 0.  1.  1.  0.]
control taken [[ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
k1 queues [ 2.  2.  0.  0.]
k2 queues [ 0.  1.  1.  0.]
control taken [[ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
k1 queues [ 2.  2.  0.  0.]
k2 queues [ 0.  1.  1.  0.]
control taken [[ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
k1 queues [ 3.  2.  0.  0.]
k2 queues [ 0.  1.  1.  0.]
control taken [[ 0.  1.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
k1 queues [ 3.  2.  0.  0.]
k2 queues [ 0.  1.  1.  0.]
control taken [[ 0.  1.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
k1 queues [ 3.  2.  0.  0.]
k2 queues [ 0.  1.  1.  0.]
control taken [[ 0.  1.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
k1 queues [ 3.  2.  0.  0.]
k2 queues [ 1.  1.  1.  0.]
control taken [[ 0.  1.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. 

In [679]:
print(interference_freq2)
print(interf_links)

[array([ 0.,  1.,  1.,  1.]), array([ 0.,  0.,  1.,  1.]), array([ 0.,  1.,  1.,  1.])]
[array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 0, 1]]), array([[0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 1]]), array([[1, 0, 1, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 1, 0, 1]])]


In [5]:
def add_arrivals(queues, arrivals):
    # for each network
    for net in range(0,len(queues)):
        for node in range(0,len(queues[net])):
            if np.random.uniform(0,1)<= arrivals[net][node]:
                queues[net][node]=queues[net][node]+1
    return queues

In [6]:
# backpressure function inputs are the 
#     state - topology of the system - which links are on / off
#     queues - lengths of queues at all nodes for each commodity
# backpressure function returns the
#     control - optimal control action - matrix of rates to send down each link per commodity and what frequency

def backpressure(state,queues_k1,queues_k2):
    # find the optimal commodity across each link
    # set to 0 if commodity 0 is optimal, 1 if commodity 1 is optimal
    # find the optimal weight across each link
    commodity=np.zeros(state.shape)
    weight=np.zeros(state.shape)
    nodes = state.shape[0]
    for start in range(0,nodes):
        for end in range(0,nodes):
            if state[start,end]>0:
                queues_k1_diff=queues_k1[start]-queues_k1[end]
                queues_k2_diff=queues_k2[start]-queues_k2[end]
                m, n=opt_com_weight(queues_k1_diff,queues_k2_diff)
                commodity[start,end]=m
                weight[start,end]=n
    
    # find all of the different possible control actions
    poss_controls = find_control_actions(state)
    
    # find the control actions that maximize sum of weights
    gains=np.zeros(len(poss_controls))
    for c in range(0,len(poss_controls)):
        gains[c]=control_gains(poss_controls[c],weight)
    
    gains=gains.tolist()
    max_value = max(gains)
    max_index = gains.index(max_value)
    control=poss_controls[max_index]
    
    return control, commodity

# returns a 0 if first commodity is bigger, 1 if second is bigger, random is equal
def opt_com_weight(com1, com2):
    if com1>com2:
        opt_com=0
        weight=max(com1,0)
    elif com2>com1:
        opt_com=1
        weight=max(com2,0)
    else:
        opt_com=np.random.choice(2)
        if opt_com==1: 
            weight=max(com1,0)
        else: 
            weight=max(com2,0)
    return opt_com, weight


# returns a list of possible control matrices
def find_control_actions(state):
    poss_controls=[]
    # list of start and end indices of on links
    start,end=np.nonzero(state)
    nonzero_index=list(range(0,len(start)))
    
    set_combos=[]
    for length in range(0,len(start)+1):
        set_combos.append(list(itertools.combinations(nonzero_index,length)))
    
    for choose in range(0,len(set_combos)):
        for j in range(0,len(set_combos[choose])):
            control=np.zeros(state.shape)
            for m in range(0,len(set_combos[choose][j])):
                k=set_combos[choose][j]
                for n in range(0,len(k)):
                    control[start[k[n]],end[k[n]]]=state[start[k[n]],end[k[n]]]
            poss_controls.append(control)
    return poss_controls

def control_gains(control, weights):
    gains=np.sum(np.multiply(control, weights))
    return gains

In [7]:
def packet_movement(queues_k1, queues_k2,control,commodity,freq,interference_freq1, interference_freq2):
    # move packets along unless there are interferences for each network
    for net in range(0,len(control)):
        # for each node, check if there's and interference!!!!
        for node1 in range(0,control[net].shape[0]):
            for node2 in range(0,control[net].shape[1]):
                # if control says send one along
                if control[net][node1,node2]>0:
                    # if transmitting on frequency, check if there's an interference
                    if (freq[net][node1,node2]==0 and interference_freq1[net][node1]==1) or (
                        freq[net][node1,node2]==0 and interference_freq1[net][node1]==1):
                        
                        # if the commodity is K1, then move K1 packet
                        if commodity[net][node1,node2]==0:
                            queues_k1[net][node1]=max(0,queues_k1[net][node1]-1)
                            queues_k1[net][node2]=queues_k1[net][node2]+1
                        # if the commodity is K2, then move K2 packet
                        else: 
                            queues_k2[net][node1]=max(0,queues_k2[net][node1]-1)
                            queues_k2[net][node2]=queues_k2[net][node2]+1
        
    # set the end of the queues to be zero - they are sinks
        queues_k1[net][-1]=0
        queues_k2[net][-1]=0
    
    return queues_k1, queues_k2
    

In [8]:
def find_interferences(control,freq,interf_links,interf_nets):
    interference_freq1=[np.ones((N[0])),np.ones((N[1])),np.ones((N[2]))]
    interference_freq2=[np.ones((N[0])),np.ones((N[1])),np.ones((N[2]))]
    for mat in range(0,len(interf_links)):
        net1=interf_nets[mat][0]
        net2=interf_nets[mat][1]
        # find the freq transmitted by nodes of each network
        for node1 in range(0,interf_links[mat].shape[0]):
            for node2 in range(0,interf_links[mat].shape[1]):
                # if there's an interference, then check if both nodes are transmitting
                if interf_links[mat][node1,node2]>0:
                    # if both nodes are on, then check frequencies
                    if sum(control[net1][node1])>0 and sum(control[net2][node2])>0:
                        # if both transmitting on the same frequency, then set interference to 0
                        node1_freq=freq[net1][node1][np.nonzero(control[net1][node1])]
                        node2_freq=freq[net2][node2][np.nonzero(control[net2][node2])]
                        for j in range(0,len(node1_freq)):
                            if node1_freq[j] in node2_freq:
                                if node1_freq[j]==0:
                                    interference_freq1[net1][node1]=0
                                    interference_freq1[net2][node2]=0
                                else:
                                    interference_freq2[net1][node1]=0
                                    interference_freq2[net2][node2]=0
    return interference_freq1, interference_freq2

In [12]:
print(freq[0])

[[1 1 0 0]
 [1 1 0 1]
 [0 0 0 1]
 [0 0 1 1]]
